In [1]:
import mmap
import sys
import numpy as np
import pandas as pd

In [2]:
%config Completer.use_jedi = False

In [17]:
TRAIN_DATASET_PATH = "/Users/ppogorelov/Downloads/dataset/ydata-ymusic-kddcup-2011-track2/trainIdx2.txt"
TEST_DATASET_PATH = "/Users/ppogorelov/Downloads/dataset/ydata-ymusic-kddcup-2011-track2/testIdx2.txt"

The dataset is distributed in unconventional format. Here is description of the format from official readme:

> First line for a user is formatted as: \<UsedId\>|\<#UserRatings\>
Each of the next \<#UserRatings\> lines describes a single rating by \<UsedId\>, sorted in chronological order.
Rating line format is: \<ItemId\>\t\<Score\>\t\<Time\>
The scores are integers lying between 0 and 100.

In [27]:
def process_file(dataset_path, dump_after_n_lines=1000000):
    with open(dataset_path, "r+b") as f:
        mm = mmap.mmap(f.fileno(), 0, prot=mmap.PROT_READ)

        interactions = []
        interactions_numpy = None

        scores = []
        scores_numpy = None

        current_user = None
        counter = 0
        line = mm.readline()
        while len(line) > 0:
            if counter % dump_after_n_lines == dump_after_n_lines-1:
                if interactions_numpy is None:
                    interactions_numpy = np.asarray(interactions)
                    scores_numpy = np.asarray(scores)
                else:
                    interactions_numpy = np.vstack((interactions_numpy, interactions))
                    scores_numpy = np.hstack((scores_numpy, scores))

                interactions = []
                scores = []

            line = line.decode("utf-8") 
            
            if '|' in line:
                current_user = int(line.split('|')[0])
            else:
                track, rating = line.split('\t')
                track = int(track)
                rating = int(rating)

                interactions.append([current_user, track])
                scores.append(rating)

            line = mm.readline()
            counter += 1
        
        
        interactions_numpy = np.vstack((interactions_numpy, interactions))
        scores_numpy = np.hstack((scores_numpy, scores))

        del interactions
        del scores
        
    return interactions_numpy, scores_numpy

In [32]:
interactions_test, scores_test = process_file(TEST_DATASET_PATH, 10000)

In [33]:
np.save("./Data/scores_test.npy", scores_test)
np.save("./Data/interactions_test.npy", interactions_test)

In [34]:
interactions_train, scores_train = process_file(TRAIN_DATASET_PATH, 1000000)

In [35]:
np.save("./Data/scores_train.npy", scores_train)
np.save("./Data/interactions_train.npy", interactions_train)